In [ ]:
import re
import os
from copy import deepcopy
from glob import glob
import datetime
import random
from collections import deque

import numpy as np

import pydicom
from pymedphys_dicom.dicom.collection import PYMEDPHYS_ROOT_UID

In [ ]:
DATA_DIR = r'S:\Physics\Programming\data\DICOM_manipulation\ct-slice-extension\original'
filepaths = glob(os.path.join(DATA_DIR, '*'))
# filepaths

In [ ]:
dicom_datasets_initial_read = [
    pydicom.dcmread(filepath, force=True)
    for filepath in filepaths
]

def instance_number(dicom_dataset):
    return dicom_dataset.InstanceNumber


dicom_datasets = deque()

for dicom_dataset in sorted(dicom_datasets_initial_read, key=instance_number):
    dicom_datasets.append(dicom_dataset)

In [ ]:
keys_to_view = ['SOPInstanceUID', 'InstanceNumber', 'SliceLocation', 'ImagePositionPatient']

def display(dicom_dataset):
    return [
        {
            key: getattr(dicom_dataset, key)
            for key in keys_to_view
        }
        for dicom_dataset in dicom_datasets
    ]
    

# display(dicom_dataset)

In [ ]:
# PYMEDPHYS_ROOT_UID

In [ ]:
def generate_UIDs(number_of_UIDs, randomisation_length=10, root=PYMEDPHYS_ROOT_UID):
    num_of_digits = len(str(number_of_UIDs))
    
    middle_item = str(random.randint(0, 10**randomisation_length)).zfill(randomisation_length)
    time_stamp_item = datetime.datetime.utcnow().strftime("%Y%m%d%H%M%S%f")
    
    last_item = [
        str(i).zfill(num_of_digits) for i in range(number_of_UIDs)
    ]
    
    UIDs = [
        '.'.join([root, middle_item, time_stamp_item, item])
        for item in last_item
    ]
    
    return UIDs

# generate_UIDs(30)

In [ ]:
# UIDs = [
#     dicom_dataset.SOPInstanceUID
#     for dicom_dataset in dicom_datasets
# ]

# # UIDs

In [ ]:
# instance_numbers = [
#     dicom_dataset.InstanceNumber
#     for dicom_dataset in dicom_datasets
# ]

# # instance_numbers

In [ ]:
# slice_locations = [
#     dicom_dataset.SliceLocation
#     for dicom_dataset in dicom_datasets
# ]

# # slice_locations

In [ ]:
# image_positions = [
#     dicom_dataset.ImagePositionPatient
#     for dicom_dataset in dicom_datasets
# ]

# # image_positions

In [ ]:
def generate_new_slice_locations(dicom_datasets, index_to_copy, number_of_slices):
    if index_to_copy == 0:
        slice_diff = dicom_datasets[0].SliceLocation - dicom_datasets[1].SliceLocation
    elif index_to_copy == len(dicom_datasets) or index_to_copy == -1:
        slice_diff = dicom_datasets[-1].SliceLocation - dicom_datasets[-2].SliceLocation
    else:
        raise ValueError('index_to_copy must be first or last slice')    
    
    new_slice_locations = [dicom_datasets[index_to_copy].SliceLocation + slice_diff]
    for _ in range(number_of_slices - 1):
        new_slice_locations.append(new_slice_locations[-1] + slice_diff)
        
    return new_slice_locations
    
    

In [ ]:
def get_append_method(dicom_datasets, index_to_copy):
    if index_to_copy == 0:
        return 'appendleft'
    elif index_to_copy == len(dicom_datasets) or index_to_copy == -1:
        return 'append'
    else:
        raise ValueError('index_to_copy must be first or last slice')    

In [ ]:
def copy_slices_and_append(dicom_datasets, index_to_copy, number_of_slices):
    append_method = get_append_method(dicom_datasets, index_to_copy)
    new_slice_locations = generate_new_slice_locations(dicom_datasets, index_to_copy, number_of_slices)
    
    dataset_to_copy = deepcopy(dicom_datasets[index_to_copy])
    
    append = getattr(dicom_datasets, append_method)
    
    for slice_location in new_slice_locations:
        new_slice = deepcopy(dataset_to_copy)
        
        new_slice.SliceLocation = str(slice_location)
        
        image_position_patient_to_copy = deepcopy(dicom_datasets[index_to_copy].ImagePositionPatient)       
        image_position_patient_to_copy[-1] = str(slice_location)
        new_slice.ImagePositionPatient = image_position_patient_to_copy
        
        append(new_slice)
        

def refresh_instance_numbers(dicom_datasets):
    for i, dicom_dataset in enumerate(dicom_datasets):
        dicom_dataset.InstanceNumber = str(i)
    
    
def generate_new_uids(dicom_datasets):
    new_UIDs = generate_UIDs(len(dicom_datasets))
    
    for dicom_dataset, UID in zip(dicom_datasets, new_UIDs):
        dicom_dataset.SOPInstanceUID = UID        
        

def extend(dicom_datasets, index_to_copy, number_of_slices):
    copy_slices_and_append(dicom_datasets, index_to_copy, number_of_slices)
    refresh_instance_numbers(dicom_datasets)
    generate_new_uids(dicom_datasets)
        
    
def extend_left(dicom_datasets, number_of_slices):
    index_to_copy = 0
    extend(dicom_datasets, index_to_copy, number_of_slices)
    
    
def extend_right(dicom_datasets, number_of_slices):
    index_to_copy = -1
    extend(dicom_datasets, index_to_copy, number_of_slices)


In [ ]:
extend_left(dicom_datasets, 50)

In [ ]:
display(dicom_dataset)

In [ ]:
common_prefix = os.path.commonprefix(filepaths)

split_prefix = common_prefix.split(os.path.sep)
assert split_prefix[-2] == 'original'
split_prefix[-2] = 'processed'

processed_common_prefix = os.path.sep.join(split_prefix)
# processed_common_prefixphysics

In [ ]:
number_of_digits = len(str(len(dicom_dataset)))

new_file_names = [
    "{}{}.dcm".format(
        processed_common_prefix, 
        str(dicom_dataset.InstanceNumber).zfill(number_of_digits)
    ) 
    for dicom_dataset in dicom_datasets
]

# new_file_names

In [ ]:
for dicom_dataset, new_file_name in zip(dicom_datasets, new_file_names):
    dicom_dataset.save_as(new_file_name)

In [ ]:
# for dcm, filename in zip(new_slices, new_ct_slice_filenames):
#     dcm.save_as(filename=filename)

In [ ]:
# structure_dcm.save_as(filename=structure_set_file)